# Mount Google Drive

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# the following code only needs to run once
# nb_path = '/content/notebooks'
# os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
# sys.path.insert(0,nb_path)

Mounted at /content/gdrive


In [3]:
!pip install --target=$nb_path torch-scatter
!pip install --target=$nb_path torch-sparse
!pip install --target=$nb_path torch-geometric

  Created wheel for torch-scatter: filename=torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl size=3225860 sha256=11fcb18faaed1a736ae1a6689a7d2e4beedb0463e43f365498c0bd9ed0e6034b
  Stored in directory: /root/.cache/pip/wheels/dd/57/a3/42ea193b77378ce634eb9454c9bc1e3163f3b482a35cdee4d1
Successfully built torch-scatter
     |████████████████████████████████| 43 kB 899 kB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.12-cp37-cp37m-linux_x86_64.whl size=1586645 sha256=56eeb49c577e0b37fff4221afc8f5001710f8d46b930dae129d2d7bc273cf9a4
  Stored in directory: /root/.cache/pip/wheels/fb/e2/2f/44956c61e3299573ffe12da9d1374c7576ca0c5fb1fe1ed38c
Successfully built torch-sparse
     |████████████████████████████████| 325 kB 6.6 MB/s 
     |████████████████████████████████| 407 kB 34.3 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.2-py3-none-any.whl size=535570 sha256=53daae6aa7c9c727174960009f9cfa8b3a81d

# Define Date Range

In [4]:
import pandas as pd
date_range = pd.date_range(start='8/1/2021', periods=2, freq='W')
date_range

DatetimeIndex(['2021-08-01', '2021-08-08'], dtype='datetime64[ns]', freq='W-SUN')

# Read-in County Dataframe

In [7]:
county_df = pd.read_csv("/content/gdrive/MyDrive/COVID_DGNN_KG/counties_static_features.csv")
county_df['FIPS'] = county_df['FIPS'].astype(str)
county_df['FIPS'] = county_df['FIPS'].str.zfill(5)

# Aggregate Mobility Data

In [8]:
import ast
import csv
import json
import numpy as np
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
from tqdm import tqdm
import sys
import os
import requests

headers = {
    'Content-Type': 'application/json',
    'apikey': 'ACMmJIIVe0CrajAPzNz7t5Ggc0jCC80k'
}

query = """
query($first: Int, $last_seen_placekey: String, $date: DateTime) {
  search(
    first: $first
    filter:{
      address: {
        region: "CA",
        iso_country_code: "US"
      }
    }
    last_seen_placekey: $last_seen_placekey
  )
  {
    placekey
    safegraph_weekly_patterns(date: $date) { 
      poi_cbg
      visitor_home_aggregation
    }
  }
}
"""

def aggregate_data(date, aggregate_result, last_seen_placekey="", first=500):
  try:
    print(date + last_seen_placekey)
    req = requests.post(
      'https://api.safegraph.com/v1/graphql',
      json={
        'query': query,
        'variables': {
          "first": first,
          "last_seen_placekey": last_seen_placekey,
          "date": date
        }
      },
      headers=headers
    )
    
    print("processing data...")
    
    r_json = req.json()
    print(r_json)
    res = r_json['data']['search']
    print("processing data...")
    for i in res:
      # Patterns
      poi_cbg = i['safegraph_weekly_patterns']['poi_cbg']
      destination_FIPS = ((str(poi_cbg)).zfill(12))[:5]
      visitor_home_aggregation = i['safegraph_weekly_patterns']['visitor_home_aggregation']
      print(visitor_home_aggregation)
      # TODO: process visitor_home_aggregation into aggregate_result
    aggregate_data(date, aggregate_result, res[-1]['placekey'])

  except Exception as e:
    print("request failed")
    print(e)

  finally:
    return



In [9]:
# loop through dates in date_range
for date in date_range:
  num_counties = len(county_df)
  aggregate_result = np.zeros(shape=(num_counties,num_counties))
  date = str(date)[:10]
  aggregate_data(date, aggregate_result, "")
  filename_local = date + ".npy"
  filename_out = join("/content/gdrive/MyDrive/COVID DGNN KG/Data/Mobility_processed/", filename_local)
  np.save(filename_out, aggregate_result)
  print("aggregation finished for: " + date)

Streaming output truncated to the last 5000 lines.
{'06095250701': 4, '06095250604': 4}
None
None
None
{'06095250200': 5, '06095251400': 5, '06095250900': 4, '06095252306': 4, '06095250105': 4}
None
{}
None
{'06095251804': 4, '06095251901': 4}
None
None
{}
None
None
{'06095252401': 4, '06055200705': 4, '06055200301': 4, '06075047702': 4, '06097150500': 4, '06055200703': 4, '06055200400': 4, '06097150100': 4, '06055201102': 4}
{'39049006220': 6, '06039000103': 5, '06075033100': 5, '06055201003': 4, '45063021027': 4, '49035113107': 4, '06059110111': 4, '06055200703': 4, '48491020110': 4, '24031703100': 4, '06073021500': 4, '18057110403': 4, '06087120500': 4, '27041450800': 4, '06041108200': 4, '06055200504': 4, '17119401721': 4, '54099020100': 4, '06055201402': 4, '21211040501': 4, '06055200602': 4, '06055201403': 4, '06085508304': 4, '06115040303': 4}
None
None
{}
None
None
None
{}
None
{'48121021519': 4, '26093742402': 4, '49035111304': 4, '06073021500': 4, '55133203804': 4, '060552007

FileNotFoundError: ignored

In [ ]:
df = np.load('/content/gdrive/MyDrive/COVID DGNN KG/Data/Mobility_processed/2020-03-01.npy')
df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])